In [1]:
import numpy as np 
import pandas as pd
import os
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# Pega todo caminho
base_dir = os.getcwd()
# Pega o caminho e atribui as variaveis
CincoCent = os.path.join(base_dir,'dataset/5') 
DezCent = os.path.join(base_dir,'dataset/10') 
VinteCincoCent = os.path.join(base_dir,'dataset/25') 
CinquentaCent = os.path.join(base_dir,'dataset/50') 
UmRealCent = os.path.join(base_dir,'dataset/100')  

# verificar se esta carreando todas as imagens
cincoCent_images = os.listdir(CincoCent)
dezCent_images = os.listdir(DezCent)
vinteCincoCent_images = os.listdir(VinteCincoCent)
cinquentaCent_images = os.listdir(CinquentaCent)
umRealCent_images = os.listdir(UmRealCent)

len(cincoCent_images), len(dezCent_images), len(vinteCincoCent_images), len(cinquentaCent_images), len(umRealCent_images)



(672, 676, 661, 640, 639)